In [2]:
%matplotlib notebook

import os
import sys
import cv2
import matplotlib.pyplot as plt
from darkflow.net.build import TFNet

ModuleNotFoundError: No module named 'cv2'

In [2]:
# directory with images from video
working_directory = '/home/analyticsuser/data/Nest Cam - Owner catches burglars in act on her phone'

# only images
files_in_directory = sorted([os.path.join(working_directory, file_name)
                      for file_name in os.listdir(working_directory)
                      if os.path.splitext(file_name)[1].lower() in ('.jpg', '.jpeg')])

# colors for different bboxes for visualization
colors = [(255,0,0), (0,255,0),(0,0,255),(255,255,0),(0,255,255),(255,0,255),(192,192,192),(128,128,128),
          (128,0,0),(128,128,0),(0,128,0),(128,0,128),(0,128,128),(0,0,128),(0,0,0)]

# configuration for YOLO
 
options = {'model': '/home/analyticsuser/darkflow/cfg/v1/yolo-tiny.cfg', 
           'load': '/home/analyticsuser/darkflow/cfg/v1/yolo-tiny.weights', 'threshold': 0.1}

#options = {'model': '/home/analyticsuser/darkflow/cfg/yolo-voc.2.0.cfg', 
#           'load': '/home/analyticsuser/darkflow/cfg/yolo-voc.2.0.weights', 'threshold': 0.1}

tfnet = TFNet(options)


/home/analyticsuser/darkflow/darkflow/dark/darknet.py:54: UserWarning: ./cfg/yolo-tiny.cfg not found, use /home/analyticsuser/darkflow/cfg/v1/yolo-tiny.cfg instead
  cfg_path, FLAGS.model))


Parsing /home/analyticsuser/darkflow/cfg/v1/yolo-tiny.cfg
Loading /home/analyticsuser/darkflow/cfg/v1/yolo-tiny.weights ...
Successfully identified 180357512 bytes
Finished in 0.02143239974975586s
Model has a VOC model name, loading VOC labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 448, 448, 3)
 Load  |  Yep!  | scale to (-1, 1)                 | (?, 448, 448, 3)
 Load  |  Yep!  | conv 3x3p1_1    leaky            | (?, 448, 448, 16)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 224, 224, 16)
 Load  |  Yep!  | conv 3x3p1_1    leaky            | (?, 224, 224, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 112, 112, 32)
 Load  |  Yep!  | conv 3x3p1_1    leaky            | (?, 112, 112, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 56, 56, 64)
 Load  |  Yep!  | conv 3x3p1_1    leaky 

In [3]:
import tensorflow as tf
import numpy as np

darknet_ckpt = tfnet.darknet

with tfnet.graph.as_default() as g:
    for var in tf.global_variables(): 
        weight = tfnet.sess.run(var)   # extract the weights
        
        # change the name of the weight and bias
        name = var.name.split(':')[0]
        var_name = name.split('-')
        name = name.replace("/", "_")
        name = name.replace("-", "_")
        name = name.replace("convolutional", "conv")
        name = name.replace("kernel", "weights")
        name = name.replace("connected", "fc")
        
        name = name.replace("4", "3")
        name = name.replace("7", "5")
        name = name.replace("10", "7")
        name = name.replace("13", "9")
        name = name.replace("16", "11")
        name = name.replace("19", "13")
        name = name.replace("21", "14")
        name = name.replace("23", "15")
        name = name.replace("26", "16")
        name = name.replace("25", "17")
        name = name.replace("30", "19")
        
        outfile = 'npy/' +name
        
        np.save(outfile, weight)
        print(outfile)
        print(var)


npy/1_conv_biases
<tf.Variable '1-convolutional/biases:0' shape=(16,) dtype=float32_ref>
npy/1_conv_weights
<tf.Variable '1-convolutional/kernel:0' shape=(3, 3, 3, 16) dtype=float32_ref>
npy/3_conv_biases
<tf.Variable '4-convolutional/biases:0' shape=(32,) dtype=float32_ref>
npy/3_conv_weights
<tf.Variable '4-convolutional/kernel:0' shape=(3, 3, 16, 32) dtype=float32_ref>
npy/5_conv_biases
<tf.Variable '7-convolutional/biases:0' shape=(64,) dtype=float32_ref>
npy/5_conv_weights
<tf.Variable '7-convolutional/kernel:0' shape=(3, 3, 32, 64) dtype=float32_ref>
npy/7_conv_biases
<tf.Variable '10-convolutional/biases:0' shape=(128,) dtype=float32_ref>
npy/7_conv_weights
<tf.Variable '10-convolutional/kernel:0' shape=(3, 3, 64, 128) dtype=float32_ref>
npy/9_conv_biases
<tf.Variable '13-convolutional/biases:0' shape=(256,) dtype=float32_ref>
npy/9_conv_weights
<tf.Variable '13-convolutional/kernel:0' shape=(3, 3, 128, 256) dtype=float32_ref>
npy/11_conv_biases
<tf.Variable '16-convolutional/bi

## testing if same results are obtained versus YOLO_tensorflow code

In [1]:
imgcv = cv2.imread('test/person_cropped.jpg')
result = tfnet.return_predict(imgcv)
print(result)

NameError: name 'cv2' is not defined

In [23]:
op = tfnet.sess.graph.get_operations()
for m in op:
    print(m.values())
    #print(m.name)
    


(<tf.Tensor 'input:0' shape=(?, 448, 448, 3) dtype=float32>,)
(<tf.Tensor 'mul/y:0' shape=() dtype=float32>,)
(<tf.Tensor 'mul:0' shape=(?, 448, 448, 3) dtype=float32>,)
(<tf.Tensor 'sub/y:0' shape=() dtype=float32>,)
(<tf.Tensor 'sub:0' shape=(?, 448, 448, 3) dtype=float32>,)
(<tf.Tensor '1-convolutional/biases/Initializer/Const:0' shape=(16,) dtype=float32>,)
(<tf.Tensor '1-convolutional/biases:0' shape=(16,) dtype=float32_ref>,)
(<tf.Tensor '1-convolutional/biases/Assign:0' shape=(16,) dtype=float32_ref>,)
(<tf.Tensor '1-convolutional/biases/read:0' shape=(16,) dtype=float32>,)
(<tf.Tensor '1-convolutional/kernel/Initializer/Const:0' shape=(3, 3, 3, 16) dtype=float32>,)
(<tf.Tensor '1-convolutional/kernel:0' shape=(3, 3, 3, 16) dtype=float32_ref>,)
(<tf.Tensor '1-convolutional/kernel/Assign:0' shape=(3, 3, 3, 16) dtype=float32_ref>,)
(<tf.Tensor '1-convolutional/kernel/read:0' shape=(3, 3, 3, 16) dtype=float32>,)
(<tf.Tensor 'Pad/paddings:0' shape=(4, 2) dtype=int32>,)
(<tf.Tensor '

In [47]:
var = op[100].values()
#print(var)


In [67]:
#var = [m for m in op if m.name == '<tf.Tensor output:0 shape=(?, 1470) dtype=float32>,'][0]  
#print(var)

var = op[181].values() #fc_17
#var = op[184].values() #fc_17
#var = op[171].values() # fc_16
print(var)
out = tfnet.sess.run(var, feed_dict)[0]
print(out)

(<tf.Tensor '28-leaky:0' shape=(?, 4096) dtype=float32>,)
[[ 0.10441415 -0.00095504  0.08513358 ...,  0.01219232 -0.01453485
   0.17149508]]


In [ ]:
import tensorflow as tf
tf.summary.FileWriter('/home/analyticsuser/yolo_graph_log/6', tfnet.sess.graph)

In [42]:
imgcv = cv2.imread('test/person_cropped.jpg')
h, w, _ = imgcv.shape
im = tfnet.framework.resize_input(imgcv) 
this_inp = np.expand_dims(im, 0)
feed_dict = {tfnet.inp : this_inp} 
out = tfnet.sess.run(tfnet.out, feed_dict)[0] 
print(out)

[ 0.0365512  -0.00983296  0.04247185 ...,  0.36868834  0.26093191
  0.36523592]


## YOLO_tensorflow output
[ 0.03655122 -0.00983297  0.04247186 ...,  0.36868834  0.26093185
     0.36523587]